In [1]:
BASE_DIR = "/Users/denis/MyProjects/Summarization"
GRAPH_BASE_DIR = "/Users/denis/MyProjects/uncased_L-12_H-768_A-12"

In [2]:
import os
import numpy as np
from bert_experimental.feature_extraction.bert_feature_extractor import BERTFeatureExtractor
from tree_rank_summaraiser import TreeRankSummariser
from data_generators.article_reference_yml_gen import ArticleReferenceGenerator
import global_const as glob_cont
import rouge
from tqdm import tqdm
from utils import evaluate
from nltk import sent_tokenize
import pandas as pd
import glob

In [3]:
GRAPH_PATH = os.path.join(GRAPH_BASE_DIR, "model_for_inference.pbtxt")
VOCAB_PATH = os.path.join(GRAPH_BASE_DIR, "vocab.txt")

encoder = BERTFeatureExtractor(GRAPH_PATH, VOCAB_PATH)
summariser = TreeRankSummariser(4, encoder)

2020-05-07 14:05:37,859 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/text_preprocessing.py:287: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

2020-05-07 14:05:37,942 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:52: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

2020-05-07 14:05:37,943 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-05-07 14:05:37,960 :  Using temporary folder as model directory: /var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmpkh8mrlag
2020-05-07 14:05:37,962 :  Using config: {'_model_dir': '/var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmpkh8mrlag', '_tf_

In [7]:
pattern_str = "/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/*"
files_list = glob.glob(pattern_str)
data_list = []
# company_name_list = ["Norwegian Cruise Line", "Norwegian Cruise", "Cruise Line"]
# company_name_list = ["Carnival"]
company_name_list = ["Pfizer"]
for path in files_list:
    print(path)
    with open(path) as file:
        article_str = file.read()
        
    personalization = {}
    sentence_list = sent_tokenize(article_str.lower())
    print(f"Total {len(sentence_list)}")
    for index,sentence in enumerate(sentence_list):
        point = 0.1
        for company_name in company_name_list:
            if company_name.lower() in sentence:
                print("Get weigth")
                point = 0.2
                break
        personalization[index] = point
    
    article_sum_list = summariser.get_summary_by_sentence(sentence_list, personalization)
    generated_sum_company_based = ". ".join(article_sum_list)
    print(len(article_sum_list))
    article_sum_list = summariser.get_summary(article_str)
    generated_sum_general = ". ".join(article_sum_list)
    print(len(article_sum_list))
    data_list.append({
        "News": article_str,
        "General summarization": generated_sum_general,
        "Company based summarization": generated_sum_company_based,
        "Are both summaries the same": generated_sum_company_based == generated_sum_general
    })
    
pd.DataFrame(data=data_list).to_excel(f"/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer.xls")

/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/Pfizer, BioNTech finish dosing in COVID-19 vax tes
Total 24
Get weigth
Get weigth
Get weigth
Get weigth
Get weigth
Get weigth
4
4
/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/GLOBAL CAR T CELL THERAPY MARKET 2019 CELLECTIS, J
Total 27
Get weigth
Get weigth
4
4
/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/Pfizer readies COVID-19 vaccine for September laun
Total 12
Get weigth
Get weigth
Get weigth
Get weigth
4
4
/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/GLOBAL DICLOFENAC MARKET OVERVIEW 2019-2025 : AURO
Total 14
Get weigth
Get weigth
4
4
/Users/denis/MyProjects/Summarization/data/Companies' news/Pfizer/Pfizer’s COVID-19 vaccine ready by September, GSK’
Total 22
Get weigth
Get weigth
Get weigth
Get weigth
Get weigth
Get weigth
4
4


In [11]:
path = "/Users/denis/MyProjects/Summarization/data/Companies' news/result_2.txt"
with open(path, mode="w") as file:
    for el in article_sum_list:
        file.write(el)
        file.write("\n")